# Diagnostics by Scenario
#### Compare diagnostic figures across scenarios using legacy reporting structure

## Change below

In [24]:
included_scenarios = {"pipelines_LNG": "NP_SSP2_6.2",
                      "HHI_0.9_gas_CHN": "NP_SSP2_6.2"}

In [2]:
run_reporting = True

## Do not change below

In [3]:
# Import packages
import os
import sys
import pandas as pd
import numpy as np
import logging
import yaml
import message_ix
import ixmp
import itertools
from datetime import datetime
    
from message_ix_models.tools.bilateralize.bilateralize import *
from message_ix_models.util import package_data_path
import message_ix_models.report.legacy.iamc_report_hackathon as legacy_reporting

from ixmp import Platform

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

[WARNING] 10:40:54 - pint.util: Redefining 'kt' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
[WARNING] 10:40:54 - pint.util: Redefining 'EUR_2005' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
[WARNING] 10:40:54 - pint.util: Redefining 'EUR' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
[DEBUG] 10:40:55 - pint.util: Changing app registry from <pint.registry.UnitRegistry object at 0x0000022B69C16510> to <pint.registry.UnitRegistry object at 0x0000022B625CD550>.
[INFO] 10:40:55 - message_ix_models.util.context: Create root Context
[DEBUG] 10:40:55 - genno.config: Override ConfigHandler(key='iamc', callback=<function iamc at 0x0000022B6B6716C0>, iterate=True, discard=True)


In [4]:
def run_legacy_reporting(scenarios_models):
    # Connect to ixmp
    mp = ixmp.Platform()
    
    # Report
    for s in scenarios_models.keys():
        scen = message_ix.Scenario(mp, model = scenarios_models[s], scenario = s)
        legacy_reporting.report(mp = mp, scen = scen)

In [5]:
if run_reporting == True:
    run_legacy_reporting(scenarios_models = included_scenarios) 

Existing index sets of 'ACTIVITY_BOUND_LO' () do not match ('node', 'technology', 'year', 'mode', 'time')
[WARNING] 10:41:07 - ixmp.model.base: Existing index sets of 'ACTIVITY_BOUND_LO' () do not match ('node', 'technology', 'year', 'mode', 'time')
Existing index sets of 'ACTIVITY_BOUND_UP' () do not match ('node', 'technology', 'year', 'mode', 'time')
[WARNING] 10:41:07 - ixmp.model.base: Existing index sets of 'ACTIVITY_BOUND_UP' () do not match ('node', 'technology', 'year', 'mode', 'time')
Existing index sets of 'ACTIVITY_BY_RATING' () do not match ('node', 'technology', 'year', 'commodity', 'level', 'time', 'rating')
[WARNING] 10:41:07 - ixmp.model.base: Existing index sets of 'ACTIVITY_BY_RATING' () do not match ('node', 'technology', 'year', 'commodity', 'level', 'time', 'rating')
Existing index sets of 'ACTIVITY_CONSTRAINT_LO' () do not match ('node', 'technology', 'year', 'time')
[WARNING] 10:41:07 - ixmp.model.base: Existing index sets of 'ACTIVITY_CONSTRAINT_LO' () do not m

processing Table: Resource|Extraction
processing Table: Resource|Cumulative Extraction
processing Table: Primary Energy


\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030

processing Table: Primary Energy (substitution method)


\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030 2030]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\report\legacy\pp_utils.py:558: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2030 2030 2030 2030 2030 2030

...

no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included


\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


no emissions included


\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
\\hdrive\home$\u045\shepard\GitHub\message-ix-models\message_ix_models\util\compat\message_data\utilities.py:191: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
\\hdrive\home$\u045\

no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included
no emissions included


In [6]:
def create_simple_interactive_chart(df,
                                    color_by, figtitle):
    """
    Simpler version using ipywidgets.interact
    """
    models = sorted(df['Model'].unique())
    scenarios = sorted(df['Scenario'].unique())
    regions = sorted(df['Region'].unique())
    
    @widgets.interact(
        model=widgets.Dropdown(options=models, value=models[0], description='Model:'),
        scenario=widgets.Dropdown(options=scenarios, value=scenarios[0], description='Scenario:'),
        region=widgets.Dropdown(options=regions, value=regions[0], description='Region:')
    )
    def plot_chart(model, scenario, region):
        # Filter data
        filtered_df = df[(df['Model'] == model) & (df['Scenario'] == scenario) & (df['Region'] == region)]
        
        if filtered_df.empty:
            print(f"No data available for Model: {model}, Scenario: {scenario}, Region: {region}")
            return
        
        # Create pivot table
        pivot_df = filtered_df.pivot_table(
            index='Year', 
            columns=color_by, 
            values='Value', 
            aggfunc='sum',
            fill_value=0
        )
        
        # Create stacked bar chart
        fig = px.bar(
            filtered_df, 
            x='Year', 
            y='Value', 
            color=color_by,
            title= figtitle + ' by ' + color_by + f' - {model}, {scenario}, {region}',
            labels={'Value': 'GWa', 'Year': 'Year'},
            color_discrete_sequence=px.colors.qualitative.D3
        )
        
        fig.update_layout(
            width=900,
            height=600,
            title_x=0.5,
            hovermode='x unified'
        )
        
        fig.update_xaxes(tickangle=45)
        fig.update_yaxes(tickformat=',.0f')
        
        fig.show()

## Import and standardize files

In [25]:
# Import files
widedf = pd.DataFrame()

for s in included_scenarios.keys():
    indf = pd.read_excel(os.path.join(
                            package_data_path("report", "legacy", "reporting_output"),
                            included_scenarios[s] + '_' + s + '.xlsx'))
    widedf = pd.concat([widedf, indf])

In [26]:
# Standardize (make long)
year_list = [y for y in widedf.columns if y not in ['Model', 'Region', 'Scenario', 'Unit', 'Variable']]

longdf = pd.DataFrame()
for y in year_list:
    df = widedf[[y, 'Model', 'Region', 'Scenario', 'Unit', 'Variable']]
    df = df.rename(columns = {y:'Value'})
    df['Year'] = int(y)
    longdf = pd.concat([longdf, df])

## 1 | Primary Energy

In [27]:
primary_energy_supply = {'Biomass': 'Primary Energy|Biomass',
                         'Coal': 'Primary Energy|Coal',
                         'Gas': 'Primary Energy|Gas',
                         'Geothermal': 'Primary Energy|Geothermal',
                         'Hydro': 'Primary Energy|Hydro',
                         'Ocean': 'Primary Energy|Ocean',
                         'Oil': 'Primary Energy|Oil',
                         'Other': 'Primary Energy|Other',
                         'Solar': 'Primary Energy|Solar',
                         'Wind': 'Primary Energy|Wind'}

In [28]:
pesdf = longdf[longdf['Variable'].isin(list(primary_energy_supply.values()))].copy()

pesdf['Fuel'] = ''
for f in primary_energy_supply.keys():
    pesdf['Fuel'] = np.where(pesdf['Variable'] == primary_energy_supply[f],
                             f, pesdf['Fuel'])

pesdf = pesdf[['Model', 'Scenario', 'Year', 'Region', 'Fuel', 'Value']]

In [29]:
create_simple_interactive_chart(pesdf, color_by = 'Fuel', figtitle = 'Primary Energy')

interactive(children=(Dropdown(description='Model:', options=('NP_SSP2_6.2',), value='NP_SSP2_6.2'), Dropdown(…

## 2 | Secondary Energy by Type

In [30]:
secondary_energy = {'Electricity': 'Secondary Energy|Electricity',
                    'Gases': 'Secondary Energy|Gases',
                    'Heat': 'Secondary Energy|Heat',
                    'Hydrogen': 'Secondary Energy|Hydrogen',
                    'Liquids': 'Secondary Energy|Liquids',
                    'Solids': 'Secondary Energy|Solids'}

In [31]:
sesdf = longdf[longdf['Variable'].isin(list(secondary_energy.values()))].copy()

sesdf['Type'] = ''
for f in secondary_energy.keys():
    sesdf['Type'] = np.where(sesdf['Variable'] == secondary_energy[f],
                             f, sesdf['Type'])

sesdf = sesdf[['Model', 'Scenario', 'Year', 'Region', 'Type', 'Value']]

In [32]:
create_simple_interactive_chart(sesdf, color_by = 'Type', figtitle = 'Secondary Energy')

interactive(children=(Dropdown(description='Model:', options=('NP_SSP2_6.2',), value='NP_SSP2_6.2'), Dropdown(…

## 3 | Electricity Generation by Source

In [33]:
electricity = {'Biomass': 'Secondary Energy|Electricity|Biomass',
               'Coal': 'Secondary Energy|Electricity|Coal',
               'Gas': 'Secondary Energy|Electricity|Gas',
               'Geothermal': 'Secondary Energy|Electricity|Geothermal',
               'Hydro': 'Secondary Energy|Electricity|Hydro',
               'Nuclear': 'Secondary Energy|Electricity|Nuclear',
               'Oil': 'Secondary Energy|Electricity|Oil',
               'Other': 'Secondary Energy|Electricity|Other',
               'Solar': 'Secondary Energy|Electricity|Solar',
#               'Storage Losses': 'Secondary Energy|Electricity|Storage Losses',
#               'Transmission Losses': 'Secondary Energy|Electricity|Transmission Losses',
               'Wind': 'Secondary Energy|Electricity|Wind'}

In [34]:
eledf = longdf[longdf['Variable'].isin(list(electricity.values()))].copy()

eledf['Source'] = ''
for f in electricity.keys():
    eledf['Source'] = np.where(eledf['Variable'] == electricity[f],
                             f, eledf['Source'])

eledf = eledf[['Model', 'Scenario', 'Year', 'Region', 'Source', 'Value']]

In [35]:
create_simple_interactive_chart(eledf, color_by = 'Source', figtitle = 'Electricity Generation')

interactive(children=(Dropdown(description='Model:', options=('NP_SSP2_6.2',), value='NP_SSP2_6.2'), Dropdown(…

## 3 | Final Energy by Sector

In [18]:
final_energy_sector = {'Industry': 'Final Energy|Industry',
                       'Residential and Commercial': 'Final Energy|Residential and Commercial',
                       'Transportation': 'Final Energy|Transportation',
                       'Non-Energy': 'Final Energy|Non-Energy'}

In [19]:
feddf = longdf[longdf['Variable'].isin(list(final_energy_sector.values()))].copy()

feddf['Sector'] = ''
for f in final_energy_sector.keys():
    feddf['Sector'] = np.where(feddf['Variable'] == final_energy_sector[f],
                             f, feddf['Sector'])

feddf = feddf[['Model', 'Scenario', 'Year', 'Region', 'Sector', 'Value']]

In [20]:
create_simple_interactive_chart(feddf, color_by = 'Sector', figtitle = 'Final Energy Demand')

interactive(children=(Dropdown(description='Model:', options=('NP_SSP2_6.2',), value='NP_SSP2_6.2'), Dropdown(…

## 4| Extraction

In [21]:
extraction = {'Coal': 'Resource|Cumulative Extraction|Coal',
              'Gas (Conventional)': 'Resource|Cumulative Extraction|Gas|Conventional',
              'Gas (Unconventional)': 'Resource|Cumulative Extraction|Gas|Unconventional',
              'Oil (Conventional)': 'Resource|Cumulative Extraction|Oil|Conventional',
              'Oil (Unconventional)': 'Resource|Cumulative Extraction|Oil|Unconventional',
              'Uranium': 'Resource|Cumulative Extraction|Uranium'}

In [22]:
extdf = longdf[longdf['Variable'].isin(list(extraction.values()))].copy()

extdf['Fuel'] = ''
for f in extraction.keys():
    extdf['Fuel'] = np.where(extdf['Variable'] == extraction[f],
                             f, extdf['Fuel'])

extdf = extdf[['Model', 'Scenario', 'Year', 'Region', 'Fuel', 'Value']]

In [23]:
create_simple_interactive_chart(extdf, color_by = 'Fuel', figtitle = 'Extraction')

interactive(children=(Dropdown(description='Model:', options=('NP_SSP2_6.2',), value='NP_SSP2_6.2'), Dropdown(…